In [244]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

## Get the News Titles

In [245]:
url = 'https://www.infowars.com/news/'
res = requests.get(url) #get the website, return request.Response object
print(res.status_code) #statu_code: return 200(found web), 404(not found)

200


In [246]:
#h3 get title, div class: article-content
soup = BeautifulSoup(res.text, 'html.parser')
us_news_div = soup.find_all('div', re.compile('article-content'))

In [253]:
title_list = []
for i in range(len(us_news_div)):
    us_news_h3 = us_news_div[i].find_all('h3', recursive=False)
    us_news_a = us_news_h3[0].find_all('a', recursive=False)
    for index, item in enumerate(us_news_a[:]):
        title = item.text.strip()
        title_list.append(title)

## Dataframe: News | URL | Content

In [248]:
title_name = ''
url_list = []
pattern = re.compile('([^\s\w]|_)+')

for i in range(len(title_list)):
    title = re.sub("[‘’]", '', title_list[i])
    #print(title)
    strippedList = pattern.sub(' ', title)
    a = strippedList.split(" ")
    empty_string = ''
    if empty_string in a:
        a = [x for x in a if x != '']
        title_name = '-'.join(a)
    else:
        title_name = '-'.join(a)
    url = 'https://www.infowars.com/'+title_name
    url_list.append(url)

In [249]:
def get_content(url):
    res_content = requests.get(url) #get the website, return request.Response object
    if(res_content.status_code == 200):
        soup2 = BeautifulSoup(res_content.text, 'html.parser')
        article = soup2.find_all('article')
        content = article[0].find_all('p')
        full_content = ""
        for index, item in enumerate(content[:]):
            content2 = item.text.strip()   
            full_content = full_content+content2
        return full_content
    else:
        return

In [250]:
content_list = []
for i in range(len(title_list)):
    content_list.append(get_content(url_list[i]))
    

In [251]:
data = {'Title':title_list,'URL':url_list,'Content':content_list}
df = pd.DataFrame(data)
df = df[['Title', 'URL', 'Content']]
df.head()

,Title,URL,Content
0,Google Debated Gaming Search Results to Bury C...,https://www.infowars.com/Google-Debated-Gaming...,"Following Donald Trump’s election win, Google ..."
1,Pickup Packed With 11 Passengers Crashes After...,https://www.infowars.com/Pickup-Packed-With-11...,A high-speed chase near the U.S.-Mexico border...
2,Ann Coulter: ‘Trump Will Be The Last Republica...,https://www.infowars.com/Ann-Coulter-Trump-Wil...,Best selling author and conservative pundit An...
3,Broward Sheriff’s Captain Who Gave ‘Stand Down...,https://www.infowars.com/Broward-Sheriffs-Capt...,"Andrew Pollack, father of Meadow Pollack who w..."
4,"After Banning Alex Jones, YouTube Still Hosts ...",https://www.infowars.com/After-Banning-Alex-Jo...,YouTube is still hosting content by race hate ...


In [252]:
df.to_csv('result.csv')